In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch
import torchvision.models as models

from loader.dataset import VizwizDataset
from IPython.core.display import HTML

# to be deleted
import nltk
from tqdm.auto import tqdm
from loader import vizwiz as vw

# %pip install nltk, ipywidgets

In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = VizwizDataset(dtype='train', ret_type='corpus', copy_img_to_mem=False, device=device, partial=100)


loading annotations into memory...
Done (t=0.28s)
creating index...
index created! imgs = 23431, anns = 100575
tokenizing caption... done!!


In [22]:
df = dataset.df.groupby('file_name').agg(captions=('caption', list)).reset_index()
df.head()

,file_name,captions
0,VizWiz_train_00000000.jpg,[ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS ...
1,VizWiz_train_00000001.jpg,[A can of Coca Cola on a counter is shown for ...
2,VizWiz_train_00000002.jpg,[A can of crushed tomatoes are on a brown surf...
3,VizWiz_train_00000003.jpg,[A white screen with a captcha that needs to b...
4,VizWiz_train_00000004.jpg,[A box for a garden light rests in someone's l...


In [49]:
print(len(df))

def test(row):
    print(row)

t = tqdm(df.iterrows(), desc='test')
for i, row in enumerate(t):
    print(row[1]['file_name'])


23


VizWiz_train_00000000.jpg
VizWiz_train_00000001.jpg
VizWiz_train_00000002.jpg
VizWiz_train_00000003.jpg
VizWiz_train_00000004.jpg
VizWiz_train_00000005.jpg
VizWiz_train_00000006.jpg
VizWiz_train_00000007.jpg
VizWiz_train_00000008.jpg
VizWiz_train_00000009.jpg
VizWiz_train_00000010.jpg
VizWiz_train_00000011.jpg
VizWiz_train_00000012.jpg
VizWiz_train_00000013.jpg
VizWiz_train_00000014.jpg
VizWiz_train_00000015.jpg
VizWiz_train_00000016.jpg
VizWiz_train_00000017.jpg
VizWiz_train_00000018.jpg
VizWiz_train_00000020.jpg
VizWiz_train_00000021.jpg
VizWiz_train_00000022.jpg
VizWiz_train_00000023.jpg



In [4]:
import json

with open('annotations/train.json', 'r') as f:
    captions = json.load(f)

In [12]:
captions['images'][2]

{'file_name': 'VizWiz_train_00000002.jpg',
 'vizwiz_url': 'https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000002.jpg',
 'id': 2,
 'text_detected': True}

In [19]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [20]:
import pickle

with open('annotations/vocab.pkl', 'rb') as f:
    v = pickle.load(f)

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")


In [5]:
tokenizer(sentences, padding=True, truncation=True, max_length=128)

{'input_ids': [[101, 2023, 7705, 19421, 7861, 8270, 4667, 2015, 2005, 2169, 7953, 6251, 102], [101, 11746, 2024, 2979, 2004, 1037, 2862, 1997, 5164, 1012, 102, 0, 0], [101, 1996, 4248, 2829, 4419, 14523, 2058, 1996, 13971, 3899, 1012, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]}

In [2]:
resnext = models.resnext101_32x8d(pretrained=True)


In [3]:
len(list(resnext.children()))

10

In [4]:
resnext.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [41]:
import torch.nn as nn

modules = list(resnext.children())[:-2]
_resnext = nn.Sequential(*modules)

for p in _resnext.parameters():
    p.requires_grad = False

#print(type(list(_resnext.children())[7]))
for c in nn.Sequential(list(_resnext.children())[6][19:], list(_resnext.children())[-1]):
    for p in c.parameters():
        p.requires_grad = True

for p in _resnext.parameters():
    print(p.requires_grad)

        
        
#list(resnext.children())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [9]:
len(list(resnext.parameters()))

314

In [47]:
import boto3
import json

from loader.model import ModelS3
from loader.images import ImageS3

key_path = 'coco/train/000000000009.jpg'
#s3 = boto3.client('s3')
#response = s3.get_object(Bucket='assistive-vision', Key=key_path)['Body'].read().decode('utf-8')
#contents = json.loads(response)

s3 = ImageS3()
contents = s3.getImage(key_path)


In [20]:
# investigating shape

import torch
import torch.nn as nn
import torchvision.models as models

from torch.utils.data import DataLoader
from torchvision import transforms

transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.RandomCrop(256),  # get 256x256 crop from random location
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = VizwizDataset(dtype='train', ret_type='corpus', copy_img_to_mem=False, transformations=transformations, device=device, partial=10)

train = DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, pin_memory=False)

loading annotations into memory...
Done (t=0.30s)
creating index...
index created! imgs = 23431, anns = 100575
tokenizing caption... done!!


In [39]:
resnext = models.resnext101_32x8d(pretrained=True)
modules = list(resnext.children())[:-2]
        
resnext = nn.Sequential(*modules)
# resize the image to allow input images of variable size
adaptive_pool = nn.AdaptiveAvgPool2d((14, 14))
linear = nn.Linear(2048, 300)

image = dataset[0][0]


In [40]:
#t = iter(train)
#for batch_idx, batch  in enumerate(t):
#    images, captions, lengths, fname, image_id = batch
#    print(images.shape)

img = image.unsqueeze(0)

outputs = resnext(img)
print('resnext::: ', outputs.shape)
outputs = adaptive_pool(outputs)
print('adaptive_pool::: ', outputs.shape)
outputs = outputs.permute(0, 2, 3, 1)
print('permute::: ', outputs.shape)
outputs = linear(outputs)
print('linear::: ', outputs.shape)


resnext:::  torch.Size([1, 2048, 8, 8])
adaptive_pool:::  torch.Size([1, 2048, 14, 14])
permute:::  torch.Size([1, 14, 14, 2048])
linear:::  torch.Size([1, 14, 14, 300])


In [1]:
import h5py

hf = h5py.File('features/coco_detections.hdf5', 'r')

In [2]:
list(hf.keys())[:40]

['100000_boxes',
 '100000_cls_prob',
 '100000_features',
 '100001_boxes',
 '100001_cls_prob',
 '100001_features',
 '100006_boxes',
 '100006_cls_prob',
 '100006_features',
 '100008_boxes',
 '100008_cls_prob',
 '100008_features',
 '100010_boxes',
 '100010_cls_prob',
 '100010_features',
 '100012_boxes',
 '100012_cls_prob',
 '100012_features',
 '100014_boxes',
 '100014_cls_prob',
 '100014_features',
 '100016_boxes',
 '100016_cls_prob',
 '100016_features',
 '100020_boxes',
 '100020_cls_prob',
 '100020_features',
 '100021_boxes',
 '100021_cls_prob',
 '100021_features',
 '100022_boxes',
 '100022_cls_prob',
 '100022_features',
 '100034_boxes',
 '100034_cls_prob',
 '100034_features',
 '100037_boxes',
 '100037_cls_prob',
 '100037_features',
 '100050_boxes']

In [3]:
import numpy as np

max_detections = 50
precomp_data = hf['100000_features']
delta = max_detections - precomp_data.shape[0]

In [4]:
if delta > 0:
    precomp_data = np.concatenate([precomp_data, np.zeros((delta, precomp_data.shape[1]))], axis=0)
elif delta < 0:
    precomp_data = precomp_data[:self.max_detections]

In [8]:
precomp_data.shape

(31, 2048)

In [4]:
hf['100034_features'][0]

array([0.        , 1.3673873 , 0.17387764, ..., 2.8576982 , 0.03271251,
       1.0031718 ], dtype=float32)